### Yelp Dataset 10Mar2025 Validations or Proof of Concepts

#### 1. Extrat Tar File

In [7]:
# !pip3 install -r requirements.txt

In [4]:
import tarfile
import os

# Define the path to the tar file and an extraction directory
tar_path = '/media/oem/onetbsamdot/datasets/yelp_10Mar2025/Yelp-JSON/Yelp JSON/yelp_dataset.tar'
extract_path = '/media/oem/onetbsamdot/datasets/yelp_10Mar2025/Yelp-JSON/Yelp JSON/'  # or another directory of your choice

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Open and extract the tar file
with tarfile.open(tar_path, 'r') as tar:
    tar.extractall(path=extract_path)

# List the extracted files to verify
print(os.listdir(extract_path))


['Yelp Dataset Documentation & ToS copy.pdf', 'yelp_dataset.tar', 'Dataset_User_Agreement.pdf', 'yelp_academic_dataset_business.json', 'yelp_academic_dataset_checkin.json', 'yelp_academic_dataset_review.json', 'yelp_academic_dataset_tip.json', 'yelp_academic_dataset_user.json']


In [8]:
# Check for spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
print(f'Spark version: {spark.version}')

25/03/11 03:04:48 WARN Utils: Your hostname, oem resolves to a loopback address: 127.0.1.1; using 192.168.5.142 instead (on interface wlp9s0)
25/03/11 03:04:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 03:04:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 3.5.5


25/03/11 03:05:02 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


#### 2. Import JSON Tables

In [ ]:
import pandas as pd
import os

# Define file paths for specific JSON files
business_file = os.path.join(extract_path, 'business.json')
review_file = os.path.join(extract_path, 'review.json')

# Load the JSON files into DataFrames
business_df = pd.read_json(business_file, lines=True)
review_df = pd.read_json(review_file, lines=True)

# Display the first few rows of each DataFrame
print("Business DataFrame:")
print(business_df.head())

print("\nReview DataFrame:")
print(review_df.head())
